In [4]:
from os.path import exists
import pandas as pd

In [7]:
df_path = 'train.csv' if exists('train.csv') else 'https://github.com/upgini/upgini/blob/main/notebooks/train.csv.zip'
df=pd.read_csv(df_path)
df = df.sample(n=19_000,random_state=0)
df['store']=df['store'].astype(str)
df['item']=df['item'].astype(str)
df['date']=pd.to_datetime(df['date'])

df.sort_values('date',inplace=True)
df.reset_index(inplace=True,drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26
